# cover_calculator_draft

Given the following for the start and end points, calculate a stratigraphic thickness between the two points:
* latitude (decimal degrees)
* longitude (decimal degrees)
* elevation (m)
* strike of bedding (RHR)
* dip of bedding

## import stuff

In [1]:
import math
import pandas as pd
import numpy as np
from calculation_functions import vincenty_inverse
from calculation_functions import compass_bearing
from calculation_functions import fisher_mean

## read in the data

The data must follow the format of the template.

In [2]:
data = pd.read_csv('covers_template.csv')
data

,hash,start_lat,start_lon,start_elev,start_strike,start_dip,end_lat,end_lon,end_elev,end_strike,end_dip
0,1,28.001819,108.799701,1159,342,41,28.003860,108.800988,1167,335,36
1,2,28.005680,108.801012,1188,258,36,28.006401,108.801845,1228,278,40
2,3,28.007272,108.800889,1197,235,38,28.007661,108.802249,1248,302,43
3,4,28.013522,108.804249,1239,304,23,28.013856,108.805201,1247,263,28
4,5,28.014832,108.805833,1243,220,34,28.015010,108.806121,1248,327,39
5,6,28.015208,108.806413,1248,327,39,28.015553,108.808079,1252,273,41
6,7,28.015316,108.808215,1259,273,41,28.015391,108.808820,1256,340,45
7,8,28.016244,108.809221,1261,306,31,28.019158,108.810916,1282,268,11
8,9,28.019514,108.811058,1276,268,11,28.020115,108.812450,1285,295,25
9,10,28.020663,108.812890,1293,295,25,28.021017,108.814922,1294,309,18


## get the vincenty distance and compass bearing

In [3]:
for i in range(len(data.index)):
    
    # vincenty distance, converted from km to m
    data.loc[i,'s'] = vincenty_inverse((data['start_lat'][i],data['start_lon'][i]),\
                                       (data['end_lat'][i],data['end_lon'][i]))
    data.loc[i,'s'] = data.loc[i,'s'] * 1000
    
    # compass bearing
    data.loc[i,'bearing'] = compass_bearing((data['start_lat'][i],data['start_lon'][i]),\
                                            (data['end_lat'][i],data['end_lon'][i]))

## get the mean bedding

In [4]:
for i in range(len(data.index)):
    
    # first convert strike to dip direction
    data.loc[i,'start_dip_direction'] = (data['start_strike'][i] + 90) % 360
    data.loc[i,'end_dip_direction'] = (data['end_strike'][i] + 90) % 360
        
    # convert dip direction and dip of bedding to a pole (perpendicular to bedding) trend and plunge
    data.loc[i,'start_pole_trend'] = (data['start_dip_direction'][i] + 180) % 360
    data.loc[i,'start_pole_plunge'] = 90 - data['start_dip'][i]
    data.loc[i,'end_pole_trend'] = (data['end_dip_direction'][i] + 180) % 360
    data.loc[i,'end_pole_plunge'] = 90 - data['end_dip'][i]  
    
    # get the mean pole
    di_block = [[data['start_pole_trend'][i],data['start_pole_plunge'][i]],\
                [data['end_pole_trend'][i],data['end_pole_plunge'][i]]]
    fpars = fisher_mean(di_block)
    data.loc[i,'mean_pole_trend'] = fpars['dec']
    data.loc[i,'mean_pole_plunge'] = fpars['inc']
    
    # convert back to mean dip direction and dip
    data.loc[i,'mean_dip_direction'] = (data['mean_pole_trend'][i] + 180) % 360
    data.loc[i,'mean_dip'] = 90 - data['mean_pole_plunge'][i]

## get the distance perpendicular to strike and the elevation difference

In [5]:
for i in range(len(data.index)):
    
    data.loc[i,'perp_s'] = data['s'][i] * math.cos(math.radians(abs(data['mean_dip_direction'][i] -\
                                                                    data['bearing'][i])))
    
    data.loc[i,'d_elev'] = data['end_elev'][i] - data['start_elev'][i]

## get the true perpendicular distance (accounting for elevation), the absolute inclination between the adjusted points, the inclination from the line between the adjusted points, and the stratigraphic height

In [6]:
for i in range(len(data.index)):
    
    data.loc[i,'R'] = math.sqrt(data['d_elev'][i]**2 + data['perp_s'][i]**2)
        
    data.loc[i,'inclination'] = math.degrees(math.atan(data['d_elev'][i] / data['perp_s'][i]))
    
    if data['d_elev'][i] > 0:
        data.loc[i,'angle'] = 90 - data['inclination'][i] - data['mean_dip'][i]
    else:
        data.loc[i,'angle'] = 90 - data['mean_dip'][i] + data['inclination'][i]
        
    data.loc[i,'HEIGHT'] = round(data['R'][i] * math.cos(math.radians(data['angle'][i])), 1)

## output data

In [7]:
data

,hash,start_lat,start_lon,start_elev,start_strike,start_dip,end_lat,end_lon,end_elev,end_strike,...,mean_pole_trend,mean_pole_plunge,mean_dip_direction,mean_dip,perp_s,d_elev,R,inclination,angle,HEIGHT
0,1,28.001819,108.799701,1159,342,41,28.003860,108.800988,1167,335,...,248.692351,51.551945,68.692351,38.448055,199.799057,8.0,199.959154,2.292911,49.259034,130.5
1,2,28.005680,108.801012,1188,258,36,28.006401,108.801845,1228,278,...,178.451551,52.423876,358.451551,37.576124,77.897445,40.0,87.567185,27.180302,25.243574,79.2
2,3,28.007272,108.800889,1197,235,38,28.007661,108.802249,1248,302,...,180.437912,54.525820,0.437912,35.474180,44.372894,51.0,67.601433,48.974880,5.550940,67.3
3,4,28.013522,108.804249,1239,304,23,28.013856,108.805201,1247,263,...,191.539857,65.913862,11.539857,24.086138,55.106565,8.0,55.684231,8.260110,57.653753,29.8
4,5,28.014832,108.805833,1243,220,34,28.015010,108.806121,1248,327,...,188.059110,66.176489,8.059110,23.823511,23.582054,5.0,24.106291,11.970892,54.205597,14.1
5,6,28.015208,108.806413,1248,327,39,28.015553,108.808079,1252,273,...,209.392732,53.215077,29.392732,36.784923,113.871126,4.0,113.941359,2.011826,51.203251,71.4
6,7,28.015316,108.808215,1259,273,41,28.015391,108.808820,1256,340,...,217.919857,52.122440,37.919857,37.877560,43.107607,-3.0,43.211871,-3.980983,48.141457,28.8
7,8,28.016244,108.809221,1261,306,31,28.019158,108.810916,1282,268,...,205.987792,69.823518,25.987792,20.176482,363.279150,21.0,363.885615,3.308403,66.515115,145.0
8,9,28.019514,108.811058,1276,268,11,28.020115,108.812450,1285,295,...,196.683902,72.398906,16.683902,17.601094,103.322926,9.0,103.714161,4.978215,67.420691,39.8
9,10,28.020663,108.812890,1293,295,25,28.021017,108.814922,1294,309,...,210.907800,68.642247,30.907800,21.357753,136.555154,1.0,136.558816,0.419572,68.222674,50.7


In [8]:
data.to_csv('covers_calculated_template.csv', index=False)